### Potential Errors to Notice

#### 1. If there is no price history data before the first book close date, there will be error regarding indexing.  so make sure there              is data in excel before the first book close date
#### 2. Also, if there is no price data before first book closure date, the adjusted price series will be missing in the initial phase dates.
#### 3. If there is error "cannot convert string i.e. 1900/1/1.....to float" then edit the right ratio data manually in excel.
#### 4. Same data repeat in cash, bonus, and rightshare
#### 5. It is important to sort the unadjusted data before fragmentation. If there's no any adjustment the raw data has to be sorted 


# PART I - Dividend and Right Data Collection

In [1]:
import pandas as pd

### Sample Stocks List

In [2]:
samples  = pd.read_csv('D:/Thesis/Sample stocks.csv')

### Historical Price

In [3]:
data = pd.read_csv(r'D:/Thesis/TodaysPrice Nov 2011-09 March 2023.csv', encoding= 'unicode_escape', low_memory= False)
data = data[data['Ticker'].isin(samples.Ticker)]
ticker_wise_data = data[['Date','Ticker','Open', 'High', 'Low', 'Close','Traded Shares']]
new_ticker_wise_data = ticker_wise_data.copy()
new_ticker_wise_data['Date'] = pd.to_datetime(new_ticker_wise_data['Date'])
new_ticker_wise_data.set_index('Date', inplace = True)
float_cols = ['Open', 'High', 'Low', 'Close']
new_ticker_wise_data[float_cols] = new_ticker_wise_data[float_cols].astype(float)

In [53]:
dividend_data = pd.read_excel('D:/Thesis/Dividend and Right History.xlsx', sheet_name= 'DividendHistory')
right_share_data = pd.read_excel('D:/Thesis/Dividend and Right History.xlsx', sheet_name= 'RightShare')

In [54]:
def cash_dividend_function(ticker):
    dividend_data_ticker_filter = dividend_data[dividend_data['Symbol'] == ticker]
    if len(dividend_data_ticker_filter) == 0: #If there is no any dividend in the specified ticker
        return pd.DataFrame(columns = ['Symbol', 'Adjustment factor','BookCloseDate', 'Adjustment Type']) #return blank df with specified columns
    else:
        dividend_data_ticker_filter = dividend_data_ticker_filter[['Symbol', 'Cash Dividend(%)', 'DividendBookClosure']].dropna() #Drop NaN rows
        dividend_data_ticker_filter.rename(columns = {'DividendBookClosure' : 'BookCloseDate', #Renaming columns to make uniform with 
                                                     'Cash Dividend(%)' : 'Adjustment factor'  # Bonus share and right share df
                                                     }, inplace = True)
        dividend_data_ticker_filter['Adjustment Type'] = 'Cash Dividend' #Column with "Cash Dividend"  
        return dividend_data_ticker_filter

In [55]:
def dividend_function(ticker):
    dividend_data_ticker_filter = dividend_data[dividend_data['Symbol'] == ticker]
    if len(dividend_data_ticker_filter) == 0: #If there is no any dividend in the specified ticker
        return pd.DataFrame(columns = ['Symbol', 'Adjustment factor','BookCloseDate', 'Adjustment Type'])
    else:
        dividend_data_ticker_filter = dividend_data_ticker_filter[['Symbol', 'Bonus Dividend(%)', 'DividendBookClosure']].dropna()
        dividend_data_ticker_filter.rename(columns = {'DividendBookClosure' : 'BookCloseDate',

                                                     'Bonus Dividend(%)' : 'Adjustment factor'
                                                     }, inplace = True)
        dividend_data_ticker_filter['Adjustment Type'] = 'Bonus Share'
        return dividend_data_ticker_filter

In [56]:
def right_ratio(ratio):    
    a = ratio.split(':')
    a = list(map(float, a)) 
    ratio = []
    for i in a:
        if float(i) ==0:
            pass
        else:
            ratio.append(float(i))
    final_ratio = str(ratio[0])+ ':' + str(ratio[1])
    return final_ratio

In [57]:
def right_function(ticker):
    right_data = right_share_data.copy()
    right_data = right_data[right_data['Symbol'] == ticker]
    if len(right_data) == 0: #If there is no any dividend in the specified ticker
        return pd.DataFrame(columns = ['Symbol', 'Adjustment factor','BookCloseDate', 'Adjustment Type'])
    else:
        right_data['Ratio'] = right_data['Ratio'].dropna().astype(str) #Remove NaN Conveting right share ratio into string
        right_data['Right_ratio'] = right_data.apply(lambda x : right_ratio(x['Ratio']), axis = 1) #Deploying right_ratio function
        right_data.rename(columns = {'RightBookClosure' : 'BookCloseDate', #Which converts excel based unwanted format to 
                                    'Right_ratio' : 'Adjustment factor' #Normal right share ratio 
                                    }, inplace = True)

        right_data['Adjustment Type'] = 'aRight Share' #Adding 'Adjustment Type' column with 'Right Share' as values.
                                        #'a is added before Right Share so that it'll be easier to sort later in cash, bonus, and right sequence.
        right_data = right_data[['Symbol', 'Adjustment factor','BookCloseDate', 'Adjustment Type'  ]]
        return right_data

In [58]:
def cash_bonus_and_right(ticker):
    three_dfs = [cash_dividend_function(ticker), dividend_function(ticker), right_function(ticker)] #Brining cash, bonus, and right df together
    if (len(three_dfs[0])==0) & (len(three_dfs[1])==0) & (len(three_dfs[2])==0): #If there's no cash, bonus, and right ever
        return pd.DataFrame(columns = ['Symbol', 'Adjustment factor','BookCloseDate', 'Adjustment Type']) #return empty df
    else: #If there's either cash, bonus, or right distribution
        data_bonus_and_right = pd.concat(three_dfs,ignore_index=True) #Concat all threee dfs to one
        #Now, sort the dataframe by book close date such that cash precedes bonus and bonus preceds right adjustment. 
        data_bonus_and_right.sort_values(by = ['BookCloseDate', 'Adjustment Type'], inplace = True, ascending= [True, False])
        data_bonus_and_right = data_bonus_and_right[data_bonus_and_right['Adjustment factor'] != 0] #Exclude 0 adjustment factor
        data_bonus_and_right[['Symbol', 'Adjustment factor','BookCloseDate', 'Adjustment Type'  ]] #Selecting only specified columns
        return data_bonus_and_right

# Part II - Price Adjustment Process

#### data_fragmentation function fragments the data into different parts based on the book close dates. If there are three book close dates in the price series, the data is fragmented into 4 parts. Number of data fragments = number of book close dates +1

In [59]:
def data_fragmentation(ticker):
    book_close_dates = []
    fragmented_data = []
    price_data = new_ticker_wise_data[new_ticker_wise_data['Ticker'] == ticker].copy() #Selecting price data for specified ticker
    price_data.sort_index(axis = 0, inplace = True) #Sorting based on index i.e. Dates
    for index, book_close_date in enumerate(df['BookCloseDate']):# for every book close dates
        if index == 0: #If it's the first book close 
            partial_data = price_data.loc[price_data.index<book_close_date]#Get price series before the book close date
            fragmented_data.append(partial_data) #Append pice series before the book close date to the fragmented data list
            book_close_dates.append(book_close_date)#Append  book close date to 'book_close_dates' list.
        elif book_close_date == list(df['BookCloseDate'])[index-1]: #If the current book close date is same as previous
            pass # Then, Do nothing
        else: #If current book close is not first book close date and not same as previous 
              #Mask for Getting price series between the previous book close and current book close date
            date_range =  (price_data.index >= closure_dates[index-1]) & (price_data.index < closure_dates[index])
            partial_data = price_data.loc[date_range] #Using the 'date_range' variable to to get the data
            fragmented_data.append(partial_data) #Append the df to 'fragmented_data' list.
#             book_close_dates.append(book_close_date) #append date to 'book_close_date'.
    last_data = price_data.loc[price_data.index>=closure_dates[index]]#Getting the price data after the last book close date
    fragmented_data.append(last_data) #appening last piece of data in fragmented data. 
    return fragmented_data

In [60]:
def final_data_df():
    concat = []
    for index, date in enumerate(df.BookCloseDate): #for each book close dates
        adj_type = df.iloc[index,3] #Getting adjustment i.e. cash, bonus, or right by accessing df dataframe. 
        adjustment_factor = df.iloc[index,1] #Getting adjustment number by accessing df dataframe. 
        book_close = df.iloc[index,2] #Getting book close date
        ticker = df.iloc[index, 0] #getting ticker
        cols_to_convert = ['Open', 'High', 'Low', 'Close'] #Columns that are to be adjusted.

        if index == 0: #If it is the first book close date
            segmented_price_series = fragmented_data[book_close_dates.index(date)].copy() #Data Before First adjustment
            # This code 'book_close_dates.index(date)' gives the position of current date in 'book_close_dates' list. 
            # book_close_dates list consists of unique (duplicate removed) book closure dates 
        else:
            if date == list(df.BookCloseDate)[index-1]: #If current book close is same as previous
                segmented_price_series = concat[index-1] #Assign segmented_price_series the value of previous concat value
            else:
                #concat previous data with current fragmented data
                segmented_price_series = pd.concat([concat[index-1], fragmented_data[book_close_dates.index(date)]])
        if adj_type == 'Cash Dividend':
            price_day_before_bookclose = list(segmented_price_series[cols_to_convert].Close)[-1] #Getting last price of the current price series
            #Cash Dividend adjustment using the formula.
            segmented_price_series[cols_to_convert] = segmented_price_series[cols_to_convert]/(1+ adjustment_factor/price_day_before_bookclose)

        elif adj_type == 'Bonus Share':
            #Stock Dividend adjustment using the formula
            segmented_price_series[cols_to_convert]/=(1+adjustment_factor/100)

        elif adj_type == 'aRight Share':
            #Splitting the right ratio by ':' and getting the % of right share to adjust.
            right_adjustment = float(adjustment_factor.split(':')[1])/float(adjustment_factor.split(':')[0])
            #Adjusting for right share by the right formula
            segmented_price_series[cols_to_convert] = (segmented_price_series[cols_to_convert]+(right_adjustment*100))/(1+right_adjustment)

        concat.append(segmented_price_series) #Append the adjusted price series to the list "concat".
    #Getting the latest adjusted price series by accessing "concat" and appending the last piece of fragmented data to the series.
    #The +1 in "book_close_dates.index(date)+1" is done because the fragmented data is +1 than the number of book close dates.
    final_data = concat[index].append(fragmented_data[book_close_dates.index(date)+1])
    return final_data

In [ ]:
cash_bonus_and_right('MBL')

## Multiple Stocks

In [ ]:
for index, ticker in enumerate(samples.Ticker):
    #Samples comprises of list of companies to be adjusted.
    print(index, ticker)
    # cash_bonus_and_right function returns the dataframe with cash, bonus, and right with book close dates in a single df.
    df = cash_bonus_and_right(ticker)
    if len(df) == 0: #If the company has not announced cash, bonus, or right
        adjusted_series = new_ticker_wise_data[new_ticker_wise_data['Ticker'] == ticker].copy() #Simply return raw price data
        adjusted_series.sort_index(axis = 0, inplace = True)
        adjusted_series.to_csv(f"D:/Thesis/adjusted prices/{ticker}.csv")
    else: #If company has announced either cash, bonus, or right 
        closure_dates = list(df['BookCloseDate']) #Book close dates duplicates
        book_close_dates = list(df.BookCloseDate.drop_duplicates()) #Book close dates without duplicates
        fragmented_data = data_fragmentation(ticker) #Calling the 'data_fragmentation' function to get fragmented data.
        adjusted_series = final_data_df() #Calling the "final_data_df" function for adjusting the price series
        adjusted_series.to_csv(f"D:/Thesis/adjusted prices/{ticker}.csv")